<a href="https://colab.research.google.com/github/FrederiKob/Kelly_Replication_Code/blob/main/Run_Rudimentary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use git to make clone of GitHub repository

In [69]:
!git clone https://github.com/FrederiKob/Kelly_Replication_Code.git

fatal: destination path 'Kelly_Replication_Code' already exists and is not an empty directory.


Draw Data from Rudimentary_Replication_Kelly_DataPrep Script

In [70]:
%run "Rudimentary_Replication_Kelly_DataPrep.ipynb"

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Import Library_Function by running script Library_Function

In [71]:
%run "Library_Function.ipynb"

# Begin Rudimentary Run

Define Parameters for a restricted and rudimentary run-through of the general model script

*   Restrict predictions to $P = 12000$ and $alpha = 1000$
*   begin and end are reduced to a few years to get a good gist of the quality of the model/similarities to Kelly et al.
*   Furthermore, faster run-through, given the reduced computational demand



In [72]:
P = 12000
alpha = 1000

# 50 should be sufficient to get a grasp of the results
max_iterations = 50

# begin and end include very volatile period (better portrayal of bheaviour of market timing positions)
begin = "1940-01-01"
end = "1960-01-01"
# determine the according index location
idx_begin = np.where(ret_std.index == begin)[0][0]
idx_end = np.where(ret_std.index == end)[0][0]

# reduce ret_std and pred_std so that data-load is reduced
ret_std = ret_std.iloc[:idx_end+1]
pred_std = pred_std.iloc[:idx_end+1,:]

Run Ridge Regression over loops (very sloppy but rudimentary)

In [73]:
np.random.seed(0)
w_i_all = generate_w_i(max_iterations=max_iterations, max_P=P)

In [ ]:
from sklearn.linear_model import Ridge

res1,res2,res3 = [],[],[]
for seed in range(max_iterations):
    pred1,pred2,pred3 = [],[],[]
    signals = generate_Signals(pred_std=pred_std, P=P, use_seed=seed, w_i_all=w_i_all)

    for t in range(idx_begin,idx_end):
        x1,y1 = Option_generate_X_y(signals=signals, ret_std=ret_std, idx_start=t, T=12, P=P, vol_stand=True)
        clf1 = Ridge(alpha=alpha, fit_intercept=False)
        # works in spyder, does not work here??
        #clf1.fit(X=x1[0],y=y1[0])
        clf1.fit(X=np.asarray(x1[0]),y=y1[0])
        # pred1.append(clf1.predict(x1[1])[0])
        pred1.append(clf1.predict(np.asarray(x1[1]))[0])

        x2,y2 = Option_generate_X_y(signals=signals, ret_std=ret_std, idx_start=t, T=60, P=P, vol_stand=True)
        clf2 = Ridge(alpha=alpha, fit_intercept=False)
        #clf2.fit(X=x2[0],y=y2[0])
        #pred2.append(clf2.predict(x2[1])[0])
        clf2.fit(X=np.asarray(x2[0]),y=y2[0])
        pred2.append(clf2.predict(np.asarray(x2[1]))[0])

        x3,y3 = Option_generate_X_y(signals=signals, ret_std=ret_std, idx_start=t, T=120, P=P, vol_stand=True)
        clf3 = Ridge(alpha=alpha, fit_intercept=False)
        #clf3.fit(X=x3[0],y=y3[0])
        #pred3.append(clf3.predict(x3[1])[0])
        clf3.fit(X=np.asarray(x3[0]),y=y3[0])
        pred3.append(clf3.predict(np.asarray(x3[1]))[0])

    print("Iteration number {} out of {} is done!".format(seed+1,max_iterations))
    res1.append(pred1)
    res2.append(pred2)
    res3.append(pred3)

Iteration number 0 out of 50 is done!
Iteration number 1 out of 50 is done!
Iteration number 2 out of 50 is done!
Iteration number 3 out of 50 is done!
Iteration number 4 out of 50 is done!
Iteration number 5 out of 50 is done!
Iteration number 6 out of 50 is done!
Iteration number 7 out of 50 is done!
Iteration number 8 out of 50 is done!
Iteration number 9 out of 50 is done!
Iteration number 10 out of 50 is done!
Iteration number 11 out of 50 is done!
Iteration number 12 out of 50 is done!
Iteration number 13 out of 50 is done!
Iteration number 14 out of 50 is done!
Iteration number 15 out of 50 is done!
Iteration number 16 out of 50 is done!
Iteration number 17 out of 50 is done!
Iteration number 18 out of 50 is done!
Iteration number 19 out of 50 is done!
Iteration number 20 out of 50 is done!
Iteration number 21 out of 50 is done!
Iteration number 22 out of 50 is done!
Iteration number 23 out of 50 is done!
Iteration number 24 out of 50 is done!
Iteration number 25 out of 50 is do

Plot the results

In [ ]:
# make sure not to overwrite resutls
results = (res1.copy(), res2.copy(), res3.copy())

# make into dataframes
plot1 = pd.DataFrame(results[0], columns=ret_std.index[idx_begin:idx_end]).T
plot2 = pd.DataFrame(results[1], columns=ret_std.index[idx_begin:idx_end]).T
plot3 = pd.DataFrame(results[2], columns=ret_std.index[idx_begin:idx_end]).T

# make dictionary --> plot_figure10 needs it (in main python script everything is handled as dicitonary)
T_plot = [12,60,120]
cmbs = list(itertools.product(T_plot,[P],[alpha]))
plot_dic = {cmbs[0]:plot1, cmbs[1]:plot2, cmbs[2]:plot3}

plot_fig10(results=plot_dic, T=T_plot, P=P, alpha=alpha, roll_window=6, begin=begin, end=end)